In [1]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [2]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.8 MB/s eta 0:00:00


In [3]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import pandas as pd

In [17]:
ROOT_DIR = '/content/drive/My Drive/'
df= pd.read_excel(ROOT_DIR + "preprocessed_arabic_data.xlsx")

In [18]:
df

,date,Text,source,Text_pro
0,2023-05-31,مبادرة الحكومة في تشجيع الشباب على ممارسة أنشط...,hespress,مبادرة الحكومة تشجيع الشباب ممارسة أنشطة المقا...
1,2023-05-31,الحكومة تعمل على تقليص الفوارق المجالية فيما ي...,hespress,الحكومة تعمل تقليص الفوارق المجالية يتعلق بجلب...
2,2023-05-31,ستنكب الحكومة بشكل مكثف على تنزيل مقتضيات القا...,hespress,ستنكب الحكومة بشكل مكثف تنزيل مقتضيات القانون ...
3,2023-05-31,نتمنى أن يحمل هذا المعرض كل ما يمكنه أن يعود ب...,hespress,نتمنى يحمل المعرض يمكنه يعود بالنفع المملكة ال...
4,2023-05-31,الحكومة المغربية ستعمل على تنزيل التوجيهات الم...,hespress,الحكومة المغربية ستعمل تنزيل التوجيهات الملكية...
...,...,...,...,...
2335,2020-04-02,اغلب الحلاقة اميين,YouTube,اغلب الحلاقة اميين
2336,2020-04-02,كترة الهم تضحك,YouTube,كترة الهم تضحك
2337,2020-04-02,صدروهم للطاليان يعدموهم,YouTube,صدروهم للطاليان يعدموهم
2338,2020-04-02,ديوه يكمليه لحسانة فجيور,YouTube,ديوه يكمليه لحسانة فجيور


In [20]:
from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModel,AutoConfig
from torch.utils.data import DataLoader, Dataset
class MyDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_seq_length=256):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        text = row["Text_pro"] #the column of the preprocessed data

        # Tokenize the text and truncate/pad to the desired sequence length
        inputs = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_seq_length,
            return_tensors='pt'
        )

        input_ids = inputs["input_ids"].squeeze()
        attention_mask = inputs["attention_mask"].squeeze()

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask
        }

In [21]:
from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModel,AutoConfig
from torch.utils.data import DataLoader, Dataset
checkpoint = "UBC-NLP/MARBERT"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
max_seq_length = 256  # Adjust as needed

# Create train dataset and dataloader
dataset = MyDataset(df, tokenizer, max_seq_length)
batch_size = 32 # Adjust as needed
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [22]:
class MyTopicPredictionModel(nn.Module):
    def __init__(self, checkpoint, num_topics):
        super(MyTopicPredictionModel, self).__init__()
        self.num_topics = num_topics

        self.model = AutoModel.from_pretrained(checkpoint, config=AutoConfig.from_pretrained(checkpoint, output_hidden_states=True))
        self.dropout = nn.Dropout(0.1)
        self.lstm = nn.LSTM(768, 256, num_layers=1, dropout=0.1, bidirectional=False, batch_first=True)
        self.classifier = nn.Linear(256, num_topics)  # Number of topics as output labels

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)

        last_hidden_state = outputs.last_hidden_state
        sequence_outputs = self.dropout(last_hidden_state)
        lstm_out, _ = self.lstm(sequence_outputs)  # Remove [:, -1, :]
        logits =F.softmax(self.classifier(lstm_out[:, -1, :]))
        #logits = self.classifier(sequence_outputs)
        #logits = F.softmax(logits)
  # Use the last output from LSTM
        return logits

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load the model
model = MyTopicPredictionModel(checkpoint='UBC-NLP/MARBERT', num_topics=7)

# Load the saved state dictionary
model.load_state_dict(torch.load('/content/drive/My Drive/model.pth'))
model.to(device)  # Ensure it's on the correct device (CPU or GPU)

# Set the model to evaluation mode
model.eval()


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


MyTopicPredictionModel(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(100000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

In [24]:

all_predicted_labels = []
with torch.no_grad():
    for batch in dataloader:  # Assuming you have a test DataLoader similar to train DataLoader
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        logits = model(input_ids=input_ids, attention_mask=attention_mask)
        predicted_labels = torch.argmax(logits, dim=1)
        all_predicted_labels.extend(predicted_labels.cpu().numpy())

<ipython-input-22-640967ae019d>:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  logits =F.softmax(self.classifier(lstm_out[:, -1, :]))


In [25]:
df['topic'] = all_predicted_labels

In [26]:
df

,date,Text,source,Text_pro,topic
0,2023-05-31,مبادرة الحكومة في تشجيع الشباب على ممارسة أنشط...,hespress,مبادرة الحكومة تشجيع الشباب ممارسة أنشطة المقا...,6
1,2023-05-31,الحكومة تعمل على تقليص الفوارق المجالية فيما ي...,hespress,الحكومة تعمل تقليص الفوارق المجالية يتعلق بجلب...,0
2,2023-05-31,ستنكب الحكومة بشكل مكثف على تنزيل مقتضيات القا...,hespress,ستنكب الحكومة بشكل مكثف تنزيل مقتضيات القانون ...,3
3,2023-05-31,نتمنى أن يحمل هذا المعرض كل ما يمكنه أن يعود ب...,hespress,نتمنى يحمل المعرض يمكنه يعود بالنفع المملكة ال...,3
4,2023-05-31,الحكومة المغربية ستعمل على تنزيل التوجيهات الم...,hespress,الحكومة المغربية ستعمل تنزيل التوجيهات الملكية...,2
...,...,...,...,...,...
2335,2020-04-02,اغلب الحلاقة اميين,YouTube,اغلب الحلاقة اميين,0
2336,2020-04-02,كترة الهم تضحك,YouTube,كترة الهم تضحك,0
2337,2020-04-02,صدروهم للطاليان يعدموهم,YouTube,صدروهم للطاليان يعدموهم,2
2338,2020-04-02,ديوه يكمليه لحسانة فجيور,YouTube,ديوه يكمليه لحسانة فجيور,6


In [27]:
label_to_topic = {
    0: 'Economy',
    1: 'Education',
    2: 'Sanitary Measures',
    3: 'Health',
    4: 'Social Life',
    5: 'Govt',
    6: 'Stats'
}

# Replace numeric labels with corresponding topics
df['topic'] = df['topic'].map(label_to_topic)

In [28]:
df

,date,Text,source,Text_pro,topic
0,2023-05-31,مبادرة الحكومة في تشجيع الشباب على ممارسة أنشط...,hespress,مبادرة الحكومة تشجيع الشباب ممارسة أنشطة المقا...,Stats
1,2023-05-31,الحكومة تعمل على تقليص الفوارق المجالية فيما ي...,hespress,الحكومة تعمل تقليص الفوارق المجالية يتعلق بجلب...,Economy
2,2023-05-31,ستنكب الحكومة بشكل مكثف على تنزيل مقتضيات القا...,hespress,ستنكب الحكومة بشكل مكثف تنزيل مقتضيات القانون ...,Health
3,2023-05-31,نتمنى أن يحمل هذا المعرض كل ما يمكنه أن يعود ب...,hespress,نتمنى يحمل المعرض يمكنه يعود بالنفع المملكة ال...,Health
4,2023-05-31,الحكومة المغربية ستعمل على تنزيل التوجيهات الم...,hespress,الحكومة المغربية ستعمل تنزيل التوجيهات الملكية...,Sanitary Measures
...,...,...,...,...,...
2335,2020-04-02,اغلب الحلاقة اميين,YouTube,اغلب الحلاقة اميين,Economy
2336,2020-04-02,كترة الهم تضحك,YouTube,كترة الهم تضحك,Economy
2337,2020-04-02,صدروهم للطاليان يعدموهم,YouTube,صدروهم للطاليان يعدموهم,Sanitary Measures
2338,2020-04-02,ديوه يكمليه لحسانة فجيور,YouTube,ديوه يكمليه لحسانة فجيور,Stats
